In [1]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("suraj520/customer-support-ticket-dataset")

print("Path to dataset files:", path)


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


100%|██████████| 828k/828k [00:00<00:00, 1.78MB/s]

Extracting files...
Path to dataset files: /home/jupyter/.cache/kagglehub/datasets/suraj520/customer-support-ticket-dataset/versions/1


In [2]:
from pathlib import Path
csv_file = next(Path(path).glob("*"))

In [3]:
import pandas as pd
df = pd.read_csv(csv_file)
df

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8464,8465,David Todd,adam28@example.net,22,Female,LG OLED,2021-12-08,Product inquiry,Installation support,My {product_purchased} is making strange noise...,Open,NaN,Low,Phone,NaN,NaN,NaN
8465,8466,Lori Davis,russell68@example.com,27,Female,Bose SoundLink Speaker,2020-02-22,Technical issue,Refund request,I'm having an issue with the {product_purchase...,Open,NaN,Critical,Email,NaN,NaN,NaN
8466,8467,Michelle Kelley,ashley83@example.org,57,Female,GoPro Action Camera,2021-08-17,Technical issue,Account access,I'm having an issue with the {product_purchase...,Closed,Eight account century nature kitchen.,High,Social media,2023-06-01 09:44:22,2023-06-01 04:31:22,3.0
8467,8468,Steven Rodriguez,fpowell@example.org,54,Male,PlayStation,2021-10-16,Product inquiry,Payment issue,I'm having an issue with the {product_purchase...,Closed,We seat culture plan.,Medium,Email,2023-06-01 18:28:24,2023-06-01 05:32:24,3.0


In [1]:
%run ../../utils/access_store_ui.ipynb
display(get_access_store_ui('../../'))

Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

In [3]:
from exasol.nb_connector.connections import open_pyexasol_connection

In [7]:
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(f"""
    CREATE OR REPLACE TABLE "{ai_lab_config.db_schema}"."ORIGINAL_CUSTOMER_SUPPORT_TICKETS" (
        "TICKET_ID" INTEGER, 
        "CUSTOMER_NAME" VARCHAR(2000000), 
        "CUSTOMER_EMAIL" VARCHAR(2000000), 
        "CUSTOMER_AGE" INTEGER, 
        "CUSTOMER_GENDER" VARCHAR(2000000), 
        "PRODUCT_PURCHASED" VARCHAR(2000000), 
        "DATE_OF_PURCHASE" VARCHAR(2000000), 
        "TICKET_TYPE" VARCHAR(2000000), 
        "TICKET_SUBJECT" VARCHAR(2000000), 
        "TICKET_DESCRIPTION" VARCHAR(2000000), 
        "TICKET_STATUS" VARCHAR(2000000), 
        "RESOLUTION" VARCHAR(2000000), 
        "TICKET_PRIORITY" VARCHAR(2000000), 
        "TICKET_CHANNEL" VARCHAR(2000000), 
        "FIRST_RESPONSE_TIME" VARCHAR(2000000), 
        "TIME_TO_RESOLUTION" VARCHAR(2000000), 
        "CUSTOMER_SATISFACTION_RATING" VARCHAR(2000000)
    );
    """)
    conn.import_from_pandas(df, table=(ai_lab_config.db_schema,"ORIGINAL_CUSTOMER_SUPPORT_TICKETS"))


In [8]:
from pandas import option_context
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    table_df = conn.export_to_pandas(f"""
        SELECT *
        FROM "{ai_lab_config.db_schema}"."ORIGINAL_CUSTOMER_SUPPORT_TICKETS"
        """)
with option_context('display.max_rows', 1000, 'display.max_columns', 20, 'max_colwidth', 1000):
    template_parameter = table_df["TICKET_DESCRIPTION"].str.extractall("(\{.+?\})").reset_index()[["level_0",0]].groupby("level_0").agg(set)
    single_template_parameter = template_parameter[template_parameter.map(len)==1].dropna()
    merge = pd.merge(table_df,single_template_parameter, left_index=True, right_index=True)
    filtered_table_df = merge[table_df.columns]
    display(filtered_table_df.head(100))

,TICKET_ID,CUSTOMER_NAME,CUSTOMER_EMAIL,CUSTOMER_AGE,CUSTOMER_GENDER,PRODUCT_PURCHASED,DATE_OF_PURCHASE,TICKET_TYPE,TICKET_SUBJECT,TICKET_DESCRIPTION,TICKET_STATUS,RESOLUTION,TICKET_PRIORITY,TICKET_CHANNEL,FIRST_RESPONSE_TIME,TIME_TO_RESOLUTION,CUSTOMER_SATISFACTION_RATING
0,4097,Steven Rodriguez,spatton@example.net,70,Male,MacBook Pro,2020-01-16,Cancellation request,Network problem,"I'm having an issue with the {product_purchased}. Please assist.\n\n""Yes, I do have it."" Harry said, leaning back a little bit. It was certainly as good a thing as it was a thing. This problem started occurring after the recent software update. I haven't made any other changes to the device.",Open,NaN,Low,Chat,NaN,NaN,NaN
2,4099,Karen Moses,ehunter@example.net,42,Female,Amazon Echo,2021-07-13,Technical issue,Product compatibility,"I'm having an issue with the {product_purchased}. Please assist.\n\nI'll probably send in an invoice, but when I figure that out I may need to ask for a refund or to send an additional order. The issue I'm facing is intermittent. Sometimes it works fine, but other times it acts up unexpectedly.",Open,NaN,Critical,Email,NaN,NaN,NaN
3,4100,Erik Hicks,thomas94@example.net,44,Other,Apple AirPods,2021-10-22,Product inquiry,Account access,"I'm having an issue with the {product_purchased}. Please assist.\n\n3. If you just want to make sure your order makes it home right away, then if your order comes on time please let me know and I will I'm not sure if this issue is specific to my device or if others have reported similar problems.",Open,NaN,Medium,Social media,NaN,NaN,NaN
4,4101,Eric James,xmartin@example.net,59,Female,Lenovo ThinkPad,2021-04-28,Technical issue,Payment issue,I'm having an issue with the {product_purchased}. Please assist. I'm concerned about the security of my {product_purchased} and would like to ensure that my data is safe.,Pending Customer Response,NaN,Medium,Chat,2023-06-01 20:54:51,NaN,NaN
5,4102,Katherine Smith,monica57@example.net,47,Female,Nintendo Switch Pro Controller,2021-04-14,Product inquiry,Product recommendation,"I'm having an issue with the {product_purchased}. Please assist.\n\nI just purchased a Nexus 5X, but I can't buy Android 6.0 Nougat or even Nexus 7 with the Play Store app. I I've followed the troubleshooting steps mentioned in the user manual, but the issue persists.",Open,NaN,Medium,Chat,NaN,NaN,NaN
6,4103,Sheila Pacheco,fdaniel@example.com,24,Female,GoPro Hero,2020-12-16,Refund request,Data loss,"I'm unable to access my {product_purchased} account. It keeps displaying an 'Invalid Credentials' error, even though I'm using the correct login information. How can I regain access to my account? Here's what you I've recently updated the firmware of my {product_purchased}, and the issue started happening afterward. Could it be related to the update?",Closed,Speech just foot.,Critical,Social media,2023-06-01 00:20:54,2023-06-01 13:26:54,5.0
7,4104,Savannah Williams,baileymiller@example.com,20,Male,Roomba Robot Vacuum,2020-06-03,Product inquiry,Hardware issue,"I'm having an issue with the {product_purchased}. Please assist.\n\n\nHow do you get a new bottle?\n\n\nWhen you register the bottle and take the check into your account, your name and the company logo will appear I've checked for software updates, and my {product_purchased} is already running the latest version.",Pending Customer Response,NaN,Low,Email,2023-06-01 12:58:55,NaN,NaN
8,4105,Sydney Chavez,kmeadows@example.net,18,Other,Dell XPS,2020-04-02,Refund request,Cancellation request,"I'm having an issue with the {product_purchased}. Please assist.\n\nThe $1,899 minimum order price is $2,799 + $2,499 shipping. So for $1,299 you will get 1 ( I've noticed a sudden decrease in battery life on my {product_purchased}. It used to last much longer.",Open,NaN,Medium,Chat,NaN,NaN,NaN
9,4106,Howard Davis,sadams@example.org,28,Male,Nintendo Switch Pro Controller,2021-12-22,Cancellation request,Battery life,"I've accidentally deleted important

In [9]:
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(f"""
    CREATE OR REPLACE TABLE "{ai_lab_config.db_schema}"."FILTERED_CUSTOMER_SUPPORT_TICKETS" (
        "TICKET_ID" INTEGER, 
        "CUSTOMER_NAME" VARCHAR(2000000), 
        "CUSTOMER_EMAIL" VARCHAR(2000000), 
        "CUSTOMER_AGE" INTEGER, 
        "CUSTOMER_GENDER" VARCHAR(2000000), 
        "PRODUCT_PURCHASED" VARCHAR(2000000), 
        "DATE_OF_PURCHASE" VARCHAR(2000000), 
        "TICKET_TYPE" VARCHAR(2000000), 
        "TICKET_SUBJECT" VARCHAR(2000000), 
        "TICKET_DESCRIPTION" VARCHAR(2000000), 
        "TICKET_STATUS" VARCHAR(2000000), 
        "RESOLUTION" VARCHAR(2000000), 
        "TICKET_PRIORITY" VARCHAR(2000000), 
        "TICKET_CHANNEL" VARCHAR(2000000), 
        "FIRST_RESPONSE_TIME" VARCHAR(2000000), 
        "TIME_TO_RESOLUTION" VARCHAR(2000000), 
        "CUSTOMER_SATISFACTION_RATING" VARCHAR(2000000)
    );
    """)
    conn.import_from_pandas(filtered_table_df, table=(ai_lab_config.db_schema,"FILTERED_CUSTOMER_SUPPORT_TICKETS"))


In [10]:
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    filtered_tickets_df = conn.export_to_pandas(f"""
    SELECT * FROM "{ai_lab_config.db_schema}"."FILTERED_CUSTOMER_SUPPORT_TICKETS"
    """)
    with option_context('display.max_rows', 10, 'display.max_columns', 20, 'max_colwidth', 1000):
        display(filtered_tickets_df)

,TICKET_ID,CUSTOMER_NAME,CUSTOMER_EMAIL,CUSTOMER_AGE,CUSTOMER_GENDER,PRODUCT_PURCHASED,DATE_OF_PURCHASE,TICKET_TYPE,TICKET_SUBJECT,TICKET_DESCRIPTION,TICKET_STATUS,RESOLUTION,TICKET_PRIORITY,TICKET_CHANNEL,FIRST_RESPONSE_TIME,TIME_TO_RESOLUTION,CUSTOMER_SATISFACTION_RATING
0,4097,Steven Rodriguez,spatton@example.net,70,Male,MacBook Pro,2020-01-16,Cancellation request,Network problem,"I'm having an issue with the {product_purchased}. Please assist.\n\n""Yes, I do have it."" Harry said, leaning back a little bit. It was certainly as good a thing as it was a thing. This problem started occurring after the recent software update. I haven't made any other changes to the device.",Open,NaN,Low,Chat,NaN,NaN,NaN
1,4099,Karen Moses,ehunter@example.net,42,Female,Amazon Echo,2021-07-13,Technical issue,Product compatibility,"I'm having an issue with the {product_purchased}. Please assist.\n\nI'll probably send in an invoice, but when I figure that out I may need to ask for a refund or to send an additional order. The issue I'm facing is intermittent. Sometimes it works fine, but other times it acts up unexpectedly.",Open,NaN,Critical,Email,NaN,NaN,NaN
2,4100,Erik Hicks,thomas94@example.net,44,Other,Apple AirPods,2021-10-22,Product inquiry,Account access,"I'm having an issue with the {product_purchased}. Please assist.\n\n3. If you just want to make sure your order makes it home right away, then if your order comes on time please let me know and I will I'm not sure if this issue is specific to my device or if others have reported similar problems.",Open,NaN,Medium,Social media,NaN,NaN,NaN
3,4101,Eric James,xmartin@example.net,59,Female,Lenovo ThinkPad,2021-04-28,Technical issue,Payment issue,I'm having an issue with the {product_purchased}. Please assist. I'm concerned about the security of my {product_purchased} and would like to ensure that my data is safe.,Pending Customer Response,NaN,Medium,Chat,2023-06-01 20:54:51,NaN,NaN
4,4102,Katherine Smith,monica57@example.net,47,Female,Nintendo Switch Pro Controller,2021-04-14,Product inquiry,Product recommendation,"I'm having an issue with the {product_purchased}. Please assist.\n\nI just purchased a Nexus 5X, but I can't buy Android 6.0 Nougat or even Nexus 7 with the Play Store app. I I've followed the troubleshooting steps mentioned in the user manual, but the issue persists.",Open,NaN,Medium,Chat,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7559,6940,Jordan Shaffer,nlogan@example.org,69,Female,Fitbit Versa Smartwatch,2020-06-15,Product inquiry,Network problem,"I'm having an issue with the {product_purchased}. Please assist. Thank you for your time and information. I am sorry, but none of the information provided is accurate.\n\nIn order to return an item to us, they I'm unable to find the option to perform the desired action in the {product_purchased}. Could you please guide me through the steps?",Pending Customer Response,NaN,Critical,Chat,2023-06-01 01:07:00,NaN,NaN
7560,6942,Amy Ochoa,sburke@example.org,28,Other,Canon EOS,2020-04-07,Technical issue,Hardware issue,"I'm having an issue with the {product_purchased}. Please assist. Thank You.\n\nQ: My purchase from the retailer came through with a faulty camera and an email, and when I received the invoice, I had to pay I've already contacted customer support multiple times, but the issue remains unresolved.",Pending Customer Response,NaN,Low,Chat,2023-06-01 20:00:03,NaN,NaN
7561,6943,Larry Garza,kevindominguez@example.com,25,Female,LG OLED,2021-12-15,Billing inquiry,Data loss,"I'm having an issue with the {product_purchased}. Please assist.\n\nPlease enter the PIN as follows:\n\nPIN = 5 (0x25)\n\nPIN_POWER = 15 (0x3D) I've followed online tutorials and community forums to troubleshoot the issue, but no luck so far.",Pending Customer Response,NaN,Medium,Social media,2023-06-01 08:29:05,NaN,NaN
7562,6944,Daniel Hansen,nstuart@example.org,26,Other,Nest Thermostat,2021-11-05,Technical issue,Product compati

In [22]:
import random

# Dictionary mapping characters to adjacent characters on a QWERTY keyboard and similar-looking characters
typo_map = {
    'a': 'qsãä',
    'b': 'vgn',
    'c': 'xdfv',
    'd': 'erfcx',
    'e': 'rdswêëε',
    'f': 'rtgvc',
    'g': 'tyhbq',
    'h': 'yujnb',
    'i': 'uojkl1íï',
    'j': 'uikmnh',
    'k': 'iolmj',
    'l': 'kop1',
    'm': 'njk',
    'n': 'bhjm',
    'o': 'ipl0óö',
    'p': 'ol;',
    'q': 'wsa',
    'r': 'edft',
    's': 'edxzaw',
    't': 'rfgy',
    'u': 'yhjiüú',
    'v': 'cfgb',
    'w': 'qase',
    'x': 'zsdc',
    'y': 'tghu',
    'z': 'xsa2',
    '0': 'oO',
    '1': 'lI!',
    '2': 'zZ',
}

def add_typos(text, typo_rate=0.05, swap_rate=0.02, lower_rate=0.03):
    """Add typos, swap characters, and randomly convert uppercase to lowercase in the given text.
    
    Args:
        text (str): The original text.
        typo_rate (float): The probability of each character getting a typo. Default is 0.05.
        swap_rate (float): The probability of swapping adjacent characters. Default is 0.02.
        lower_rate (float): The probability of converting uppercase to lowercase. Default is 0.03.
        
    Returns:
        str: The text with added typos, swaps, and lowercase conversions.
    """
    new_text = list(text)  # Convert to list for easy manipulation
    i = 0
    while i < len(new_text):
        # Convert uppercase to lowercase
        if new_text[i].isupper() and random.random() < lower_rate:
            new_text[i] = new_text[i].lower()

        # Handle typo substitution
        if new_text[i].lower() in typo_map and random.random() < typo_rate:
            typo_char = random.choice(typo_map[new_text[i].lower()])
            if new_text[i].isupper():
                typo_char = typo_char.upper()
            new_text[i] = typo_char

        # Handle swapping of adjacent characters, skip if any is space
        if (
            i < len(new_text) - 1 and
            new_text[i] != ' ' and new_text[i + 1] != ' ' and
            random.random() < swap_rate
        ):
            # Swap current character with the next one
            new_text[i], new_text[i + 1] = new_text[i + 1], new_text[i]
            # Move past swapped characters
            i += 1

        i += 1

    return ''.join(new_text)

typo_tickets_df=filtered_tickets_df
typo_tickets_df["PRODUCT_PURCHASED_TYPO"]=typo_tickets_df["PRODUCT_PURCHASED"].map(lambda x: add_typos(x,0.001))
typo_tickets_df["TICKET_DESCRIPTION"]=typo_tickets_df.apply(lambda row: row['TICKET_DESCRIPTION'].replace("{product_purchased}", row['PRODUCT_PURCHASED_TYPO']), axis=1)
with option_context('display.max_rows', 10, 'display.max_columns', 20, 'max_colwidth', 1000):
    typo_tickets_df=typo_tickets_df[~ (typo_tickets_df["TICKET_DESCRIPTION"].str.contains("{") | typo_tickets_df["TICKET_DESCRIPTION"].str.contains("}"))]
    typo_tickets_df=typo_tickets_df.drop(columns="PRODUCT_PURCHASED_TYPO")
    display(typo_tickets_df)

NameError: name 'filtered_tickets_df' is not defined

In [12]:
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(f"""
    CREATE OR REPLACE TABLE "{ai_lab_config.db_schema}"."CUSTOMER_SUPPORT_TICKETS" (
        "TICKET_ID" INTEGER, 
        "CUSTOMER_NAME" VARCHAR(2000000), 
        "CUSTOMER_EMAIL" VARCHAR(2000000), 
        "CUSTOMER_AGE" INTEGER, 
        "CUSTOMER_GENDER" VARCHAR(2000000), 
        "PRODUCT_PURCHASED" VARCHAR(2000000),
        "DATE_OF_PURCHASE" VARCHAR(2000000), 
        "TICKET_TYPE" VARCHAR(2000000), 
        "TICKET_SUBJECT" VARCHAR(2000000), 
        "TICKET_DESCRIPTION" VARCHAR(2000000), 
        "TICKET_STATUS" VARCHAR(2000000), 
        "RESOLUTION" VARCHAR(2000000), 
        "TICKET_PRIORITY" VARCHAR(2000000), 
        "TICKET_CHANNEL" VARCHAR(2000000), 
        "FIRST_RESPONSE_TIME" VARCHAR(2000000), 
        "TIME_TO_RESOLUTION" VARCHAR(2000000), 
        "CUSTOMER_SATISFACTION_RATING" VARCHAR(2000000)
    );
    """)
    conn.import_from_pandas(typo_tickets_df, table=(ai_lab_config.db_schema,"CUSTOMER_SUPPORT_TICKETS"))


In [12]:
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    tickets_df = conn.export_to_pandas(f"""
    SELECT TICKET_ID, CUSTOMER_NAME, DATE_OF_PURCHASE, TICKET_SUBJECT, TICKET_DESCRIPTION, TICKET_STATUS, TICKET_CHANNEL FROM "{ai_lab_config.db_schema}"."CUSTOMER_SUPPORT_TICKETS"
    """)
    display(tickets_df)

,TICKET_ID,CUSTOMER_NAME,DATE_OF_PURCHASE,TICKET_SUBJECT,TICKET_DESCRIPTION,TICKET_STATUS,TICKET_CHANNEL
0,4097,Steven Rodriguez,2020-01-16,Network problem,I'm having an issue with the MacBook Pro. Plea...,Open,Chat
1,4099,Karen Moses,2021-07-13,Product compatibility,I'm having an issue with the Amaozn Echo. Plea...,Open,Email
2,4100,Erik Hicks,2021-10-22,Account access,I'm having an issue with the pAple AirPosd. Pl...,Open,Social media
3,4101,Eric James,2021-04-28,Payment issue,I'm having an issue with the Lenovo ThinkPad. ...,Pending Customer Response,Chat
4,4102,Katherine Smith,2021-04-14,Product recommendation,I'm having an issue with the Nintendo Switch P...,Open,Chat
...,...,...,...,...,...,...,...
7347,5387,Sharon Dyer,2020-01-09,Display issue,I'm having an issue with the Microsoft Surface...,Open,Chat
7348,5388,Shannon Cameron,2021-05-13,Product compatibility,I'm having an issue with the GoPro Hero. Pleas...,Closed,Email
7349,5389,Jonathan Chang,2021-04-11,Battery life,I'm having an issue with the Microsoft Office....,Pending Customer Response,Email
7350,5390,David Holloway,2021-01-18,Product compatibility,I'm having an issue with the Ftibit Charge. Pl...,Pending Customer Response,Phone


In [13]:
tickets_df.to_csv("customer_support_tickets.csv", index=False)

In [15]:
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    result = conn.execute(f"""
    CREATE OR REPLACE TABLE "{ai_lab_config.db_schema}"."PRODUCTS" AS
    SELECT row_number() over () as PRODUCT_ID, PRODUCT_PURCHASED as PRODUCT_NAME
    FROM (
        SELECT distinct PRODUCT_PURCHASED FROM "{ai_lab_config.db_schema}"."CUSTOMER_SUPPORT_TICKETS"
    )
    """)

In [4]:
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    result = conn.export_to_pandas(f"""
    SELECT * FROM "{ai_lab_config.db_schema}"."PRODUCTS"
   
    """)
    display(result)

,PRODUCT_ID,PRODUCT_NAME
0,1,Nikon D
1,2,Microsoft Xbox Controller
2,3,GoPro Hero
3,4,Adobe Photoshop
4,5,PlayStation
5,6,Amazon Kindle
6,7,Lenovo ThinkPad
7,8,Samsung Galaxy
8,9,Canon DSLR Camera
9,10,Xbox


In [7]:
result.to_csv("products.csv", index=False)